In [1]:
library("tidyverse")
library("data.table")
library("cowplot")
library("ggh4x")
library("ggrastr")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘data.table’


The following objects are masked from ‘package:lubridate’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose



Attaching package: ‘cowplot’


The following object is masked from ‘package:lubridate’:

    stamp




In [3]:
df <- fread("/nfs/research/birney/users/saul/nextflow/medaka_behaviour_pilot/hmm/time_step0.08_n_states15_hmm.csv.gz")
df[, mean_dist := log10(mean(distance)), by = hmm_state]
tmp <- df[, .(hmm_state, mean_dist)] |> distinct() |> as.data.table()
tmp[, hmm_state_recoded := rank(mean_dist)]
df <- merge(df, tmp, by = c("hmm_state", "mean_dist"))
df[, hmm_state := hmm_state_recoded]
df[, hmm_state_recoded := NULL]
df <- df |>
    separate(
        id,
        into = c("date", "time_string", "ref_strain", "test_strain", "tank_side", "assay", "quadrant", "fish_type"),
        sep = "_",
        remove = FALSE
    ) |>
    as.data.table()
df[, test_strain := fct_recode(test_strain, !!!strain_labels)]
df[, test_strain := fct_relevel(test_strain, function(x){names(strain_labels)})]
head(df)

hmm_state,mean_dist,id,date,time_string,ref_strain,test_strain,tank_side,assay,quadrant,fish_type,frame_n,time_s,distance,angle
<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<fct>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>
6,0.1005448,20190611_1331_icab_icab_R_no_q1_ref,20190611,1331,icab,iCab,R,no,q1,ref,757,25.23333,0.2083191,0.0000000
6,0.1005448,20190611_1331_icab_icab_R_no_q1_ref,20190611,1331,icab,iCab,R,no,q1,ref,761,25.36667,0.4425455,0.0000000
6,0.1005448,20190611_1331_icab_icab_R_no_q1_ref,20190611,1331,icab,iCab,R,no,q1,ref,1755,58.50000,1.2756671,-0.4597774
6,0.1005448,20190611_1331_icab_icab_R_no_q1_ref,20190611,1331,icab,iCab,R,no,q1,ref,1779,59.30000,1.3794776,-0.1900450
6,0.1005448,20190611_1331_icab_icab_R_no_q1_ref,20190611,1331,icab,iCab,R,no,q1,ref,1781,59.36667,1.3745458,-0.3463498
6,0.1005448,20190611_1331_icab_icab_R_no_q1_ref,20190611,1331,icab,iCab,R,no,q1,ref,1787,59.56667,1.4936201,-0.3414281


In [4]:
run_kw <- function(the_state, the_assay, data){
    fit <- kruskal.test(f ~ test_strain, data = data[assay == the_assay & hmm_state == the_state])
    ret <- data.table(
        assay = the_assay,
        hmm_state = the_state,
        pval = fit[["p.value"]],
        pval = fit[["p.value"]]
    )
    return(ret)
}

In [6]:
# NOTE: here we run each id independently differently from fig7!
summ <- df[
    , .(id = str_remove(id, "_(ref|test)$"), test_strain, fish_type, assay, frame_n, hmm_state)
]
summ <- dcast(summ, id + test_strain + assay + frame_n ~ fish_type, value.var = "hmm_state")[!is.na(test) & !is.na(ref)]
summ <- summ[, .(n = .N), by = c("ref", "test", "test_strain", "assay", "id")]
summ <- summ[, .(test, ref, n, n_tot = sum(n)), by = c("test_strain", "assay", "id")]
summ[, f := n/n_tot]
summ <- summ[ref == test, .(test_strain, assay, hmm_state = test, f)]
head(summ)

test_strain,assay,hmm_state,f
<fct>,<chr>,<dbl>,<dbl>
iCab,no,1,0.035341187
iCab,no,4,0.015114470
iCab,no,7,0.005779062
iCab,no,2,0.006001334
iCab,no,3,0.006668148
iCab,no,6,0.007890642


In [8]:
res <- c(
    lapply(summ[, unique(hmm_state)], run_kw, the_assay = "of", data = summ),
    lapply(summ[, unique(hmm_state)], run_kw, the_assay = "no", data = summ)
) |>
    rbindlist()

In [9]:
res[, q := p.adjust(pval, method = "fdr")]
res[, sig := q < 0.05]

In [10]:
res[, .(n = .N, min = min(q) |> signif(3), max = max(q) |> signif(3)), by = c("sig", "assay")]

sig,assay,n,min,max
<lgl>,<chr>,<int>,<dbl>,<dbl>
TRUE,of,7,1.68e-07,0.0483
FALSE,of,8,2.82e-01,0.7830
TRUE,no,5,2.54e-07,0.0483
FALSE,no,10,9.10e-02,0.8800


In [18]:
pretty_df <- res[
    , .(
        `Assay component` = ifelse(assay == "of", "Open field", "Novel object"),
        `HMM State` = hmm_state,
        `p-value (FDR-adjusted)` = signif(q, 3) |> format(scientific = TRUE)
    )
][order(`HMM State`), .SD, by = `Assay component`]

pretty_df

Assay component,HMM State,p-value (FDR-adjusted)
<chr>,<dbl>,<chr>
Open field,1,1.68e-07
Open field,2,1.30e-02
Open field,3,8.03e-03
Open field,4,7.98e-06
Open field,5,6.85e-01
Open field,6,5.42e-01
Open field,7,1.30e-02
Open field,8,2.82e-01
Open field,9,3.69e-01


In [19]:
fwrite(pretty_df, "tableS3.csv")